# Disc Golf Coefficients Optimizer
#### This document calculates all the local maximum of disc golf throws using various inital conditions
#### The goal of this research is to see if mathmatical models of disc golf throws can simulate reality (throw 800+ ft). Current simulations seem to indicate that conditions only allow for shorter throws.

In [1]:
from ipynb.fs.full.sharedfunctions import compute_custom_trajectory, Result

In [2]:
def to_tuple(a):
    amt = a*0.1
    return (a-amt,a+amt)

In [3]:
PL0 = to_tuple(0.33)
PLa = to_tuple(1.9)
PD0 = to_tuple(0.18)
PTy0 = to_tuple(-0.082)
PTya = to_tuple(0.43)
x = (-5,5)
y = (-5,5)
z = (1,5)
vx = (10,60)
vy = (-5,5)
vz = (-5,5)
phi = (-1,1)
theta = (-1,1)
gamma = (-1,1)
dphi = (-1,1)
dtheta = (-1,1)
dgamma = to_tuple(62.0)
alpha_0  = to_tuple(-4*3.14/180)
PDa = to_tuple(0.69)
PTxwx = to_tuple(-0.013)
PTxwz = to_tuple(-0.0017)
PTywy = to_tuple(-0.014)
PTzwz = to_tuple(-0.000034)
area = to_tuple(0.058556)
I_zz = to_tuple(0.002352)
I_xx = to_tuple(0.001219)
mass = to_tuple(0.175)

In [4]:
all_vars = [
    PL0,
    PLa,
    PD0,
    PTy0,
    PTya,
    #x,
    #y,
    #z,
    vx,
    vy,
    vz,
    phi,
    theta,
    gamma,
    dphi,
    dtheta,
    dgamma,
    alpha_0,
    PDa,
    PTxwx,
    PTxwz,
    PTywy,
    PTzwz,
    area,
    I_zz,
    I_xx,
    mass,
]

In [5]:
number_of_iterations_per_var = 10

In [6]:
def compute_custom_trajectory_optimizer(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = 0,
        PDa = None,
        PTxwx = None,
        PTxwz = None,
        PTywy = None,
        PTzwz = None,
        area = None,  # m^2
        I_zz = None,  # kg*m^2
        I_xx = None,  # kg*m^2
        mass = None,  # kg
    ):
    result = compute_custom_trajectory(
        PL0,
        PLa,
        PD0,
        PTy0,
        PTya,
        x,
        y,
        z,
        vx,
        vy,
        vz,
        phi,
        theta,
        gamma,
        dphi,
        dtheta,
        dgamma,
        alpha_0 = alpha_0,
        PDa = PDa,
        PTxwx = PTxwx,
        PTxwz = PTxwz,
        PTywy = PTywy,
        PTzwz = PTzwz,
        area = area,
        I_zz = I_zz,
        I_xx = I_xx,
        mass = mass
    )
    if result is not None and len(result.x) > 0 and len(result.y) > 0:
        return result.x[-1], result.y[-1]
    return 0,0

In [7]:

def getResults(variables, index = 0, values = []):
    global count
    #if count > 50:
    #    return []
    #print("getResults index = "+str(index)+" values = "+str(values))
    subresults = []
    var = variables[index]
    range_of_var = var[1] - var[0]
    for itteration in range(number_of_iterations_per_var+1):
        if index == len(variables)-3:
            print("getResults itteration = "+str(itteration))
        value = var[0] + range_of_var * (itteration/number_of_iterations_per_var)
        new_values = []
        for v in values:
            new_values.append(v)
        new_values.append(value)
        if index >= len(variables) - 1:
            end_x, end_y = compute_custom_trajectory_optimizer(
                new_values[0],
                new_values[1],
                new_values[2],
                new_values[3],
                new_values[4],
                0, # x
                0, # y
                1, # z
                new_values[5],
                new_values[6],
                new_values[7],
                new_values[8],
                new_values[9],
                new_values[10],
                new_values[11],
                new_values[12],
                new_values[13],
                new_values[14],
                new_values[15],
                new_values[16],
                new_values[17],
                new_values[18],
                new_values[19],
                new_values[20],
                new_values[21],
                new_values[22],
                new_values[23],
            )
            subresults.append(Result(new_values, end_x, end_y))
            count += 1
        else:
            for subresult in getResults(variables,index + 1, new_values):
                subresults.append(subresult)
    return subresults

In [8]:
count = 0
results = getResults(all_vars)

getResults itteration = 0
getResults itteration = 1
getResults itteration = 2
getResults itteration = 3
getResults itteration = 4
getResults itteration = 5
getResults itteration = 6
getResults itteration = 7
getResults itteration = 8
getResults itteration = 9
getResults itteration = 10


In [9]:
with open('coefficients_optimizer_results.txt', 'w') as f:
    for item in results:
        f.write(f'{item.toString()}\n')